In [1]:
!pip install jsonref


[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [50]:
import requests
import json

def parse_spec(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

openapi = parse_spec('v4-sgp-spec-07-04-2024.json')

url = 'http://127.0.0.1:8000/query'

# Define the body of the request
data = {
    "query": "I want to create a question set with 1 questions about tennis. create the question set and question. use the account id '6630377a5a7b09c735cfeebb'. for post requests remember to input the body",
    "headers": {"x-api-key": "c5ae00cd-6190-41cd-802e-b2d848b11fb8"},
    "base_url": "https://api.egp.scale.com",
    "openapi": openapi  # Assuming an empty dictionary for the openapi key, update this accordingly if needed
}

# Convert the dictionary to a JSON-formatted string
json_data = json.dumps(data)

# Set headers to indicate JSON content type
headers = {
    'Content-Type': 'application/json'
}

# Make the POST request
# response = requests.post(url, data=json_data, headers=headers)

with requests.post(url, data=json_data, headers=headers, stream=True) as response:
    if response.status_code == 200:
        # Iterate over lines in the response
        for line in response.iter_lines():
            if line:  # Filter out keep-alive new lines
                decoded_line = line.decode('utf-8')
                print(decoded_line)
    else:
        print("Failed to connect to the server:", response.status_code)

{"type":"info","message":"Agent successfully started","metadata":{}}
{"type":"tool_in","message":"Running tool: get_dependency_tree","metadata":{}}
{"type":"tool_out","message":"Tool complete: get_dependency_tree","metadata":{"tool_call_id":"call_ajwoP898UH69TW2PO2ZkrVJe","output":"Dependency tree for question-set:\nquestion-set\n└── question\n"}}
{"type":"tool_in","message":"Running tool: get_dependency_tree","metadata":{}}
{"type":"tool_out","message":"Tool complete: get_dependency_tree","metadata":{"tool_call_id":"call_krCFfj70zBP09A6S9xSvKApR","output":"Dependency tree for question:\nquestion\n"}}
{"type":"tool_in","message":"Running tool: fetch_api_spec","metadata":{}}
{"type":"tool_out","message":"Tool complete: fetch_api_spec","metadata":{"tool_call_id":"call_VfXL10dMcMjwAr0Sb9aP9X19","output":"Route: post '/v4/question-sets'\nDescription:\n### Description\nCreates a question set\n\n### Details\nThis API can be used to create a question set. To use this API, review the request s

In [2]:
import jsonref

def parse_spec(file_path):
    with open(file_path, 'r') as file:
        return jsonref.load(file)

In [3]:
openapi = parse_spec('v4-sgp-spec-07-04-2024.json')
openapi

{'openapi': '3.1.0',
 'info': {'title': 'EGP API V4',
  'description': 'This is the parent API for all EGP APIs. If you are looking for the EGP API, please go to https://api.egp.scale.com/docs.',
  'version': '0.1.0'},
 'servers': [{'url': 'https://api.egp.scale.com'}],
 'paths': {'/v4/knowledge-bases': {'get': {'tags': ['Knowledge Bases'],
    'summary': 'List Knowledge Bases',
    'description': '### Description\nLists all knowledge bases owned by the authorized user.\n\n### Details\nThis API can be used to list all knowledge bases that have been created by the user.         This API will return the details of all knowledge bases including their IDs, names, the         embedding models they use, any metadata associated with the knowledge bases, and the         timestamps for their creation, last-updated time.\n\n#### Backwards Compatibility\nV2 and V1 Knowledge Bases are entirely separate and not backwards compatible. Users who         have existing V1 knowledge bases will need to mi

In [33]:
# route will be input of RAG
route = "/v4/evaluations" # hardest one

naive = openapi["paths"][route]
naive

{'get': {'tags': ['Evaluations'],
  'summary': 'List Evaluations',
  'description': '### Description\nLists all evaluations accessible to the user.\n\n### Details\nThis API can be used to list evaluations. If a user has access to multiple accounts, all evaluations from all accounts the user is associated with will be returned.',
  'operationId': 'GET-V4-/evaluations',
  'parameters': [{'required': False,
    'schema': {'items': {'type': 'string',
      'enum': ['ApplicationSpec',
       'AsyncJobs',
       'EvaluationConfig',
       'TestCaseResults'],
      'title': 'EvaluationViewsOptions',
      'description': 'An enumeration.'},
     'type': 'array'},
    'name': 'view',
    'in': 'query'},
   {'required': False,
    'schema': {'items': {'enum': ['status:asc',
       'status:desc',
       'application_spec_id:asc',
       'application_spec_id:desc',
       'application_spec:asc',
       'application_spec:desc',
       'application_variant_id:asc',
       'application_variant_id:des

In [29]:
def parse_properties(properties):
    lines = []
    for key, value in properties.items():
        title = value.get('title', key.capitalize())  # Fallback to key if title is not present
        type_info = value.get('type', 'Unknown Type')
        lines.append(f"{title} ({key}): {type_info}")
    return "\n".join(lines)

def parse_schema(schema):
    output = []
    if "properties" in schema:
        title = schema.get('title', 'Untitled Schema')
        properties = parse_properties(schema['properties'])
        output.append(f"{title}\n{properties}\n")
    elif "oneOf" in schema or "allOf" in schema:
        key = "oneOf" if "oneOf" in schema else "allOf"
        for s in schema[key]:
            output.append(parse_schema(s))
    return "\n".join(output)

def schema_to_string(schema):
    return parse_schema(schema)

In [37]:
for method, details in naive.items():
    if method in ["post"]:
        request_body_json = details.get("requestBody")
        schema = request_body_json['content']['application/json']['schema']
        print(schema)
        description = details.get("description", None)
        print(description)
        schema_str = schema_to_string(schema)
        print(schema_str)
        

{'oneOf': [{'properties': {'name': {'type': 'string', 'title': 'Name'}, 'description': {'type': 'string', 'title': 'Description'}, 'application_spec_id': {'type': 'string', 'title': 'Application Spec Id'}, 'application_variant_id': {'type': 'string', 'title': 'Application Variant Id'}, 'tags': {'type': 'object', 'title': 'Tags'}, 'evaluation_config': {'type': 'object', 'title': 'Evaluation Config'}, 'evaluation_config_id': {'type': 'string', 'title': 'Evaluation Config Id', 'description': 'The ID of the associated evaluation config.'}, 'account_id': {'type': 'string', 'title': 'Account Id', 'description': 'The ID of the account that owns the given entity.', 'can_patch': False}, 'type': {'type': 'string', 'enum': ['builder'], 'const': 'builder', 'title': 'Type', 'description': 'create standalone evaluation or build evaluation which will auto generate test case results', 'default': 'builder'}, 'evaluation_dataset_id': {'type': 'string', 'title': 'Evaluation Dataset Id'}, 'evaluation_data